# Bedrock AgentCore Gateway를 사용하여 Smithy API를 MCP Tool로 변환하기

## 개요
Bedrock AgentCore Gateway는 고객이 기존 Smithy API를 인프라나 호스팅 관리 없이 완전 관리형 MCP 서버로 전환할 수 있는 방법을 제공합니다. 고객은 Smithy 사양을 가져와서 MCP Tool로 변환할 수 있습니다. 이 튜토리얼에서는 Amazon S3의 Smithy Model에서 MCP Tool을 생성하는 방법을 시연합니다. Agent는 Amazon S3를 쿼리하고 관련 질문에 답변할 수 있게 됩니다.

Gateway 워크플로우는 Agent를 외부 Tool에 연결하기 위해 다음 단계를 포함합니다:
* **Gateway용 Tool 생성** - Smithy 사양을 사용하여 Tool을 정의합니다.
* **Gateway 엔드포인트 생성** - 인바운드 인증을 사용하여 MCP 진입점 역할을 할 Gateway를 생성합니다.
* **Gateway에 타겟 추가** - Gateway가 특정 Tool로 요청을 라우팅하는 방법을 정의하는 Smithy 타겟을 구성합니다. Smithy 파일의 일부인 모든 작업은 MCP 호환 Tool이 되며 Gateway 엔드포인트 URL을 통해 사용할 수 있게 됩니다. Smithy를 통해 Amazon S3 API를 호출하기 위해 AWS IAM을 사용하여 아웃바운드 권한 부여를 구성합니다.
* **Agent 코드 업데이트** - 통합 MCP 인터페이스를 통해 구성된 모든 Tool에 액세스하도록 Agent를 Gateway 엔드포인트에 연결합니다.

![How does it work](images/smithy-apis-gateway.png)

### 튜토리얼 세부 정보


| 정보                  | 세부 사항                                                   |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형          | 인터랙티브                                                  |
| AgentCore 구성 요소   | AgentCore Gateway, AgentCore Identity                     |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target 유형  | Smithy                                                   |
| Agent                | Infrastructure agent                                        |
| Inbound Auth IdP     | Amazon Cognito                                            |
| Outbound Auth        | IAM                                                   |
| LLM model            | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| 튜토리얼 구성 요소      | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 범용                                                       |
| 예제 복잡도           | 쉬움                                                       |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 몇 가지 AmazonCore Gateway 타겟을 생성합니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 Amazon S3의 Smithy 사양에 정의된 작업을 MCP Tool로 변환하고 Bedrock AgentCore Gateway에서 호스팅합니다. 이를 통해 사용자는 자신의 AWS 계정에 있는 Amazon S3와 관련된 질문을 할 수 있습니다.

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+ 버전의 Jupyter notebook
* uv
* AWS 자격 증명
* Amazon Cognito

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
import os
# Set AWS credentials if not using SageMaker notebook
# os.environ['AWS_ACCESS_KEY_ID'] = '' # Set the access key
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # Set the secret key
# AWS_REGION 환경변수가 없으면 기본값으로 us-east-1 사용
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    # Jupyter 환경에서는 __file__이 정의되지 않으므로 현재 작업 디렉토리 사용
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
# 상위 디렉토리 2단계 위로 이동하여 utils.py 경로 설정
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add to sys.path
# Python 모듈 검색 경로에 utils 디렉토리 추가
sys.path.insert(0, utils_dir)

# Now you can import utils
import utils

In [ ]:
#### Create an IAM role for the Gateway to assume
import utils

# Gateway가 S3 Smithy API를 호출할 수 있도록 IAM 역할 생성
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role_s3_smithy("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway에 대한 인바운드 권한 부여를 위한 Amazon Cognito Pool 생성

In [ ]:
# Creating Cognito User Pool 
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
# OAuth 2.0 스코프 정의: Gateway에 대한 읽기/쓰기 권한
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
# 스코프 문자열 형식: resource-server-id/scope-name
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Creating or retrieving Cognito resources...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"User Pool ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("Resource server ensured.")

# Machine-to-Machine (M2M) 클라이언트 생성 (client credentials grant 방식)
client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"Client ID: {client_id}")

# Get discovery URL  
# OIDC discovery endpoint: JWT 토큰 검증에 필요한 공개키 및 설정 정보 제공
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Gateway 생성

In [ ]:
# CreateGateway with Cognito authorizer without CMK. Use the Cognito user pool created in the previous step
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
# Custom JWT authorizer 설정: Cognito에서 발급한 JWT 토큰으로 인증
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # Client MUST match with the ClientId configured in Cognito. Example: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoS3Smithyv3',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',  # Model Context Protocol 사용
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with Smithy target'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Amazon S3 사양을 사용하여 Smithy 타겟 생성

Amazon S3 API의 Smithy Gateway 타겟을 생성할 것입니다. Amazon S3 Smithy JSON 파일을 가져와서 API를 MCP Tool로 변환합니다.

#### S3에 S3 Smithy API JSON 파일 업로드

In [ ]:
# Create an S3 client
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# Retrieve AWS account ID and region
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# Define parameters
bucket_name = f'agentcore-gateway-{account_id}-{region}' # Your s3 bucket to upload the OpenAPI json file.
file_path = 'smithy-specs/s3-apis.json'
object_key = 's3-apis.json'
# Upload the file using put_object and read response
try:
    try:
        # us-east-1 리전은 LocationConstraint 파라미터 없이 버킷 생성
        if region == "us-east-1":
            s3bucket = s3_client.create_bucket(
                Bucket=bucket_name
            )
        else:
            s3bucket = s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': region
                }
            )
    except Exception as e:
        print(e)
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=file_data)

    # Construct the ARN of the uploaded object with account ID and region
    smithy_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'Uploaded object S3 URI: {smithy_s3_uri}')
except Exception as e:
    print(f'Error uploading file: {e}')

#### Gateway 타겟 생성

In [ ]:
# S3 Uri for Smithy spec file
# Smithy 모델 파일 위치를 MCP 타겟 설정에 지정
smithy_s3_target_config = {
    "mcp": {
          "smithyModel": {
              "s3": {
                  "uri": smithy_s3_uri
              }
          }
      }
}

# IAM credentials provider configuration
# Gateway의 IAM 역할을 사용하여 S3 API 호출 시 인증
credential_config  = {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }

targetname='DemoSmithytargetForS3'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Smithy Target with S3Uri using SDK',
    targetConfiguration=smithy_s3_target_config,
    credentialProviderConfigurations=[credential_config])

# Printing the request ID and timestamp for you to report the defects. Please include them while reporting issues/defects  
response_metadata = response['ResponseMetadata']
print(response_metadata)

# Strands Agent에서 Bedrock AgentCore Gateway 호출하기

Strands Agent는 Model Context Protocol (MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS Tool과 원활하게 통합됩니다. 이 통합은 AI Agent와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본 MCP API인 ListTools 및 InvokeTools를 노출하는 프로토콜 호환 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트 또는 SDK가 안전하고 표준화된 방식으로 사용 가능한 Tool을 검색하고 상호 작용할 수 있습니다. Strands Agent가 AWS 서비스에 액세스해야 할 때 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP Authorization 사양)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격하게 준수하여 강력한 보안 및 액세스 제어를 보장합니다. 즉, Strands Agent의 모든 Tool 호출은 권한 부여 단계를 거쳐 보안을 유지하면서 강력한 기능을 제공합니다.

예를 들어, Strands Agent가 MCP Tool에 액세스해야 할 때 먼저 ListTools를 호출하여 사용 가능한 Tool을 검색한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트 또는 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI Agent 통합을 위한 다재다능하고 미래 지향적인 솔루션입니다.

# 인바운드 권한 부여를 위해 Amazon Cognito에서 액세스 토큰 요청

In [ ]:
print("Requesting the access token from Amazon Cognito authorizer...May fail for some time till the domain name propogation completes")
# Cognito에서 OAuth 2.0 client credentials grant 방식으로 액세스 토큰 요청
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("Token response:", token)

# AWS 계정에 있는 S3 리소스에 대해 IT Agent에게 질문하기

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

# Streamable HTTP transport 생성: Gateway URL과 Bearer 토큰으로 인증
def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

# MCP 클라이언트 초기화
client = MCPClient(create_streamable_http_transport)

## The IAM group/user/ configured in ~/.aws/credentials should have access to Bedrock model
# Amazon Nova Pro 모델 설정
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # Call the listTools 
    # MCP 프로토콜의 listTools API 호출: Gateway에서 사용 가능한 모든 Tool 목록 조회
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
    #print(f"Tools loaded in the agent are {agent.tool_names}")
    #print(f"Tools configuration in the agent are {agent.tool_config}")
    # Invoke the agent with the sample prompt. This will only invoke  MCP listTools and retrieve the list of tools the LLM has access to. The below does not actually call any tool.
    agent("Hi , can you list all the s3 buckets. Locate the right tool and call the tool to get the answer")
    #agent("What is the weather in northern part of the mars")
    # Invoke the agent with sample prompt, invoke the tool and display the response
    #agent("Check the order status for order id 123 and show me the exact response from the tool")
    #Call the MCP tool explicitly. The MCP Tool name and arguments must match with your AWS Lambda function or the OpenAPI/Smithy API
    # MCP Tool 직접 호출: Tool 이름은 'targetname___APIOperationName' 형식
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # You can replace this with unique identifier. 
    name=targetname+"___ListBuckets", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    #arguments={"ver": "1.0","feedtype": "json"}
    )
    #Print the MCP Tool response
    print(f"Tool Call result: {result['content'][0]['text']}")


# 정리
IAM 역할, IAM 정책, Credentials provider, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성되므로 정리 작업의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## Gateway 삭제 (선택 사항)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)